### This project was build on top of the following repo: https://github.com/jwyang/faster-rcnn.pytorch
### Modified for thyroid nodule detection

###### --------------------------------------------------------
###### Pytorch multi-GPU Faster R-CNN
###### Licensed under The MIT License [see LICENSE for details]
###### Written by Jiasen Lu, Jianwei Yang, based on code from Ross Girshick
###### --------------------------------------------------------

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import sys
import argparse

import torch.optim as optim
import torchvision.transforms as transforms
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.faster_rcnn.vgg16 import vgg16

In [2]:
# passed
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import time
import os
import pprint
from matplotlib import pyplot as plt
import PIL.Image
import pdb
import glob
from torch.utils.cpp_extension import CUDA_HOME
db_cache = glob.glob('/home/martin/JupyterLab/data/cache/*.pkl')
if len(db_cache):
    os.remove(db_cache[0])

# Configs
from easydict import EasyDict

args = EasyDict()
args.dataset = 'pascal_voc'
args.net = 'res101'
args.large_scale = False
args.cuda = True
args.batch_size = 2
args.save_dir = '/home/martin/JupyterLab/output/faster_rcnn'
args.num_workers = 0
args.class_agnostic = True
args.lr = 0.001
args.optimizer = 'sgd'
args.resume = False
args.mGPUs = False
args.use_tfboard = False
args.start_epoch = 0
args.max_epochs = 1
args.lr_decay_step = 8
args.lr_decay_gamma = 0.333
args.disp_interval = 100
args.session = 1
args.imdb_name = "voc_2007_trainval"
args.imdbval_name = "voc_2007_test"
args.set_cfgs = ['ANCHOR_SCALES', '[8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]', 'MAX_NUM_GT_BOXES', '5']

print('Called with args:')
pprint.pprint(args)

# mute large_scale variable
args.cfg_file = 'cfgs/{}.yml'.format(args.net)

from model.utils.config import cfg, cfg_from_file
if args.cfg_file is not None:
    cfg_from_file(args.cfg_file)

if torch.cuda.is_available() and not args.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with cuda")

cfg.CUDA = args.cuda
cfg.USE_GPU_NMS = args.cuda

# train set
# -- Note: Use validation set and disable the flipped to enable faster loading.
cfg.TRAIN.USE_FLIPPED = True
cfg.POOLING_MODE = 'align'

# output dir
output_dir = args.save_dir + "/" + args.net + "/" + args.dataset
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(output_dir)

Called with args:
{'batch_size': 2,
 'class_agnostic': True,
 'cuda': True,
 'dataset': 'pascal_voc',
 'disp_interval': 100,
 'imdb_name': 'voc_2007_trainval',
 'imdbval_name': 'voc_2007_test',
 'large_scale': False,
 'lr': 0.001,
 'lr_decay_gamma': 0.333,
 'lr_decay_step': 8,
 'mGPUs': False,
 'max_epochs': 1,
 'net': 'res101',
 'num_workers': 0,
 'optimizer': 'sgd',
 'resume': False,
 'save_dir': '/home/martin/JupyterLab/output/faster_rcnn',
 'session': 1,
 'set_cfgs': ['ANCHOR_SCALES',
              '[8, 16, 32]',
              'ANCHOR_RATIOS',
              '[0.5,1,2]',
              'MAX_NUM_GT_BOXES',
              '5'],
 'start_epoch': 0,
 'use_tfboard': False}
/home/martin/JupyterLab/output/faster_rcnn/res101/pascal_voc


/home/martin/JupyterLab/faster_rcnn_v1/model/utils/config.py:376: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [3]:
# DB helper
def prepare_roidb(imdb):
    """Enrich the imdb's roidb by adding some derived quantities that
    are useful for training. This function precomputes the maximum
    overlap, taken over ground-truth boxes, between each ROI and
    each ground-truth box. The class with maximum overlap is also
    recorded.
    """
    roidb = imdb.roidb
    if not (imdb.name.startswith('coco')):
        sizes = [PIL.Image.open(imdb.image_path_at(i)).size
             for i in range(imdb.num_images)]

    for i in range(len(imdb.image_index)):
        roidb[i]['img_id'] = imdb.image_id_at(i)
        roidb[i]['image'] = imdb.image_path_at(i)
        if not (imdb.name.startswith('coco')):
            roidb[i]['width'] = sizes[i][0]
            roidb[i]['height'] = sizes[i][1]
        # need gt_overlaps as a dense array for argmax
        gt_overlaps = roidb[i]['gt_overlaps'].toarray()
        # max overlap with gt over classes (columns)
        max_overlaps = gt_overlaps.max(axis=1)
        # gt class that had the max overlap
        max_classes = gt_overlaps.argmax(axis=1)
        roidb[i]['max_classes'] = max_classes
        roidb[i]['max_overlaps'] = max_overlaps
        # sanity checks
        # max overlap of 0 => class should be zero (background)
        zero_inds = np.where(max_overlaps == 0)[0]
        assert all(max_classes[zero_inds] == 0)
        # max overlap > 0 => class should not be zero (must be a fg class)
        nonzero_inds = np.where(max_overlaps > 0)[0]
        assert all(max_classes[nonzero_inds] != 0)

def get_training_roidb(imdb):
    """Returns a roidb (Region of Interest database) for use in training."""
    if cfg.TRAIN.USE_FLIPPED:
        print('Appending horizontally-flipped training examples...')
        imdb.append_flipped_images()
        print('done')
    
    print('Preparing training data...')
    
    prepare_roidb(imdb)
    #ratio_index = rank_roidb_ratio(imdb)
    print('done')
    
    return imdb.roidb

# USE THYROID VOC
from datasets.thyroid_voc import pascal_voc
imdb = pascal_voc('trainval', '2007', '/home/martin/JupyterLab/data/VOCThyroid')

print('Loaded dataset `{:s}` for training'.format(imdb.name))

# data augmentation: 1x data -> 4x data
imdb.set_proposal_method(cfg.TRAIN.PROPOSAL_METHOD)
print('Set proposal method: {:s}'.format(cfg.TRAIN.PROPOSAL_METHOD))
roidb = get_training_roidb(imdb)

train_size = len(roidb)
print('{:d} roidb entries'.format(len(roidb)))


Loaded dataset `voc_2007_trainval` for training
Set proposal method: gt
Appending horizontally-flipped training examples...
wrote gt roidb to /home/martin/JupyterLab/data/cache/voc_2007_trainval_gt_roidb.pkl
done
Preparing training data...
done
5554 roidb entries


In [4]:
def filter_roidb(roidb):
    # filter the image without bounding box.
    print('before filtering, there are %d images...' % (len(roidb)))
    i = 0
    while i < len(roidb):
        if len(roidb[i]['boxes']) == 0:
            del roidb[i]
            i -= 1
        i += 1
    
    print('after filtering, there are %d images...' % (len(roidb)))
    return roidb

def rank_roidb_ratio(roidb):
    # rank roidb based on the ratio between width and height.
    ratio_large = 2 # largest ratio to preserve.
    ratio_small = 0.5 # smallest ratio to preserve.    
    
    ratio_list = []
    for i in range(len(roidb)):
        width = roidb[i]['width']
        height = roidb[i]['height']
        ratio = width / float(height)

        if ratio > ratio_large:
            roidb[i]['need_crop'] = 1
            ratio = ratio_large
        elif ratio < ratio_small:
            roidb[i]['need_crop'] = 1
            ratio = ratio_small        
        else:
            roidb[i]['need_crop'] = 0

        ratio_list.append(ratio)

    ratio_list = np.array(ratio_list)
    ratio_index = np.argsort(ratio_list)
    return ratio_list[ratio_index], ratio_index

from torch.utils.data.sampler import Sampler
class sampler(Sampler):
    def __init__(self, train_size, batch_size):
        self.num_data = train_size
        self.num_per_batch = int(train_size / batch_size)
        self.batch_size = batch_size
        self.range = torch.arange(0,batch_size).view(1, batch_size).long()
        self.leftover_flag = False
        if train_size % batch_size:
            self.leftover = torch.arange(self.num_per_batch*batch_size, train_size).long()
            self.leftover_flag = True

    def __iter__(self):
        rand_num = torch.randperm(self.num_per_batch).view(-1,1) * self.batch_size
        self.rand_num = rand_num.expand(self.num_per_batch, self.batch_size) + self.range

        self.rand_num_view = self.rand_num.view(-1)

        if self.leftover_flag:
            self.rand_num_view = torch.cat((self.rand_num_view, self.leftover),0)

        return iter(self.rand_num_view)

    def __len__(self):
        return self.num_data

sampler_batch = sampler(train_size, args.batch_size)

roidb = filter_roidb(roidb)
ratio_list, ratio_index = rank_roidb_ratio(roidb)

from roi_data_layer.roibatchLoader import roibatchLoader
from importlib import reload
import sys
roibatchLoader=reload(sys.modules['roi_data_layer.roibatchLoader']).roibatchLoader
dataset = roibatchLoader(
    roidb, 
    ratio_list, 
    ratio_index, 
    args.batch_size, 
    imdb.num_classes, 
    training=True
)

dataloader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=args.batch_size,
    sampler=sampler_batch, 
    num_workers=args.num_workers
)

before filtering, there are 5554 images...
after filtering, there are 5554 images...


In [6]:
from model.faster_rcnn.resnet import resnet

# initilize the network here.
fasterRCNN = resnet(imdb.classes, 101, pretrained=True, class_agnostic=args.class_agnostic)

fasterRCNN.create_architecture()

Loading pretrained weights from data/pretrained_model/resnet101_caffe.pth


In [32]:
###########################################################################
##### Load from checkpoint
###########################################################################
input_dir = '/home/martin/JupyterLab/output/faster_rcnn/res101/pascal_voc/'
load_name = input_dir + 'faster_rcnn_1_73_2776.pth'

print("load checkpoint %s" % (load_name))
if args.cuda > 0:
    checkpoint = torch.load(load_name)
args.session = checkpoint['session']
fasterRCNN.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
lr = optimizer.param_groups[0]['lr']
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']
    
print('load model successfully!')

fasterRCNN.cuda()
print('Ship faster RCNN to cuda')

load checkpoint /home/martin/JupyterLab/output/faster_rcnn/res101/pascal_voc/faster_rcnn_1_73_2776.pth
load model successfully!
Ship faster RCNN to cuda


In [20]:
# Specify class information
pascal_classes = np.asarray(['__background__', 'lesion'])

# initilize the tensor holder here.
im_data = torch.FloatTensor(1)
im_info = torch.FloatTensor(1)
num_boxes = torch.LongTensor(1)
gt_boxes = torch.FloatTensor(1)

# ship to cuda
if args.cuda > 0:
    im_data = im_data.cuda()
    im_info = im_info.cuda()
    num_boxes = num_boxes.cuda()
    gt_boxes = gt_boxes.cuda()

# make variable
im_data = Variable(im_data, volatile=True)
im_info = Variable(im_info, volatile=True)
num_boxes = Variable(num_boxes, volatile=True)
gt_boxes = Variable(gt_boxes, volatile=True)

fasterRCNN.eval()

start = time.time()
max_per_image = 100
thresh = 0.05
vis = True

/home/martin/JupyterLab/faster_rcnn_v1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
/home/martin/JupyterLab/faster_rcnn_v1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()
/home/martin/JupyterLab/faster_rcnn_v1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/martin/JupyterLab/faster_rcnn_v1/venv/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


In [21]:
import cv2
from model.utils.blob import im_list_to_blob
from model.rpn.bbox_transform import clip_boxes
from model.roi_layers import nms
from model.rpn.bbox_transform import bbox_transform_inv
from model.utils.net_utils import save_net, load_net, vis_detections
from matplotlib.pyplot import imread

def _get_image_blob(im):
    """Converts an image into a network input.
    Arguments:
    im (ndarray): a color image in BGR order
    Returns:
    blob (ndarray): a data blob holding an image pyramid
    im_scale_factors (list): list of image scales (relative to im) used
      in the image pyramid
    """
    im_orig = im.astype(np.float32, copy=True)
    im_orig -= cfg.PIXEL_MEANS

    im_shape = im_orig.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])

    processed_ims = []
    im_scale_factors = []

    for target_size in cfg.TEST.SCALES:
        im_scale = float(target_size) / float(im_size_min)
        # Prevent the biggest axis from being more than MAX_SIZE
        if np.round(im_scale * im_size_max) > cfg.TEST.MAX_SIZE:
              im_scale = float(cfg.TEST.MAX_SIZE) / float(im_size_max)
        im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale,
                interpolation=cv2.INTER_LINEAR)
        im_scale_factors.append(im_scale)
        processed_ims.append(im)

    # Create a blob to hold the input images
    blob = im_list_to_blob(processed_ims)

    return blob, np.array(im_scale_factors)

def vis_detections(im, class_name, dets, thresh=0.8):
    """Visual debugging of detections."""
    for i in range(np.minimum(10, dets.shape[0])):
        bbox = tuple(int(np.round(x)) for x in dets[i, :4])
        score = dets[i, -1]
        if score > thresh:
            cv2.rectangle(im, bbox[0:2], bbox[2:4], (0, 204, 0), 2)
            cv2.putText(im, '%s: %.3f' % (class_name, score), (bbox[0], bbox[1] + 15), cv2.FONT_HERSHEY_PLAIN,
                        1.0, (0, 0, 255), thickness=1)
    return im

def get_gt_bbox(img_filename):
    # e.g. '117.png'
    annotation_path = '/home/martin/JupyterLab/data/VOCThyroid/VOC2007/Annotations/'
    annotation_file = annotation_path + img_filename + '.xml'
    
    import xml.etree.ElementTree as ET
    tree = ET.parse(annotation_file)
    objs = tree.findall('object')
    
    bbox_list = []
    for ix, obj in enumerate(objs):
        bbox = obj.find('bndbox')
        # Make pixel indexes 0-based
        x1 = int(bbox.find('xmin').text)
        y1 = int(bbox.find('ymin').text)
        x2 = int(bbox.find('xmax').text)
        y2 = int(bbox.find('ymax').text)
        bbox_list.append({
            'bbox': (x1,y1,x2,y2),
            'text': 'gt'
        })
        
    return(bbox_list)

def get_pred_bbox(class_name, dets, thresh=0.8):
    # class_name = pascal_classes[j]
    # dets = cls_dets.cpu().numpy()
    # thresh = 0.3
    
    bbox_list = [] 
    # Get bbox for score > threshold
    for i in range(np.minimum(10, dets.shape[0])):
        bbox = tuple(int(np.round(x)) for x in dets[i, :4])
        score = dets[i, -1]
        if score > thresh:
            bbox_list.append({
                'bbox': bbox,
                'score': score, 
                'text': '%s: %.3f' % (class_name, score) 
            })
    
    # If no score > threshold, get bbox for highest score
    if len(bbox_list) == 0:
        dets = dets[dets[:,-1].argsort()]
        bbox = tuple(int(np.round(x)) for x in dets[-1, :4])
        score = dets[-1, -1]
        bbox_list.append({
            'bbox': bbox,
            'score': score, 
            'text': '%s: %.3f' % (class_name, score) 
        })
    
    return(bbox_list)

def vis_bbox(im, bbox_list, color = (255, 0, 0)):
    for bbox_obj in bbox_list:
        bbox = bbox_obj['bbox']
        text = bbox_obj['text']
        
        cv2.rectangle(im, bbox[0:2], bbox[2:4], color, 2)
        cv2.putText(im, text, (bbox[0], bbox[1] + 15), cv2.FONT_HERSHEY_PLAIN,
                    1.0, color, thickness=1)
    
    return im

def get_iou(bb1_raw, bb2_raw):
    bb1 = {
        'x1': bb1_raw[0],
        'y1': bb1_raw[1],
        'x2': bb1_raw[2],
        'y2': bb1_raw[3],
    }
    
    bb2 = {
        'x1': bb2_raw[0],
        'y1': bb2_raw[1],
        'x2': bb2_raw[2],
        'y2': bb2_raw[3],
    }
    
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [23]:
# DEMO MODE: LOAD IMG FROM DIR
args.image_dir = '/home/martin/JupyterLab/data/VOCThyroid/VOC2007/JPEGImages/'

# Load test.txt and combine into '[ID].png'
f_path = '/home/martin/JupyterLab/data/VOCThyroid/VOC2007/ImageSets/Main/'
f = open(f_path + 'test.txt', 'r')
imglist = f.readlines()
f.close()
imglist = list(map(lambda x: str(int(x)) + '.png', imglist))

imglist = imglist

In [33]:
######################### Loop code ###########################
iou_list = []
vis = 1
num_images = len(imglist)
print('Loaded Photo: {} images.'.format(num_images))

for i in range(num_images):
    
    total_tic = time.time()
    num_images -= 1
    
    im_file = os.path.join(args.image_dir, imglist[num_images])
    # im = cv2.imread(im_file)
    im_in = np.array(imread(im_file))
    # rgb -> bgr
    im = im_in[:,:,::-1]
    # im_scales is scale ratio
    blobs, im_scales = _get_image_blob(im)
    assert len(im_scales) == 1, "Only single-image batch implemented"
    
    im_blob = blobs
    # im_info_np is height, width, scale ratio
    im_info_np = np.array([[im_blob.shape[1], im_blob.shape[2], im_scales[0]]], dtype=np.float32)
    
    # convert to pytorch and permute color channel
    im_data_pt = torch.from_numpy(im_blob)
    im_data_pt = im_data_pt.permute(0, 3, 1, 2)
    im_info_pt = torch.from_numpy(im_info_np)
    
    with torch.no_grad():
        im_data.resize_(im_data_pt.size()).copy_(im_data_pt)
        im_info.resize_(im_info_pt.size()).copy_(im_info_pt)
        gt_boxes.resize_(1, 1, 5).zero_()
        num_boxes.resize_(1).zero_()
        
    det_tic = time.time()
    
    rois, cls_prob, bbox_pred, \
    rpn_loss_cls, rpn_loss_box, \
    RCNN_loss_cls, RCNN_loss_bbox, \
    rois_label = fasterRCNN(im_data, im_info, gt_boxes, num_boxes)
    
    # Here: box = drop first column of rois 
    scores = cls_prob.data
    boxes = rois.data[:, :, 1:5]
    
    # Apply bounding-box regression deltas
    box_deltas = bbox_pred.data
    if cfg.TRAIN.BBOX_NORMALIZE_TARGETS_PRECOMPUTED:
    # Optionally normalize targets by a precomputed mean and stdev
        if args.class_agnostic:
            if args.cuda > 0:
                box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                           + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
            else:
                box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS) \
                           + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS)
    
            box_deltas = box_deltas.view(1, -1, 4)
        else:
            if args.cuda > 0:
                box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                           + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
            else:
                box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS) \
                           + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS)
            box_deltas = box_deltas.view(1, -1, 4 * len(pascal_classes))
    
    pred_boxes = bbox_transform_inv(boxes, box_deltas, 1)
    pred_boxes = clip_boxes(pred_boxes, im_info.data, 1)
    
    pred_boxes /= im_scales[0]
    
    scores = scores.squeeze()
    pred_boxes = pred_boxes.squeeze()
    det_toc = time.time()
    detect_time = det_toc - det_tic
    misc_tic = time.time()
    if vis:
        im2show = np.copy(im)
        # This pixel*255 operation is replaced by cv2.convertScaleAbs(im2show, alpha=(255.0))
        #im2show = im2show*255
    
    ####
    ## bbox gen
    ####
    # Loop through pascal classes and inference
    for j in range(1, len(pascal_classes)):
        # get all indices of score of j class exceeding threshold
        inds = torch.nonzero(scores[:,j]>thresh).view(-1)
        
        # visuzalize gt box
        img_filename = (imglist[num_images]).replace('.png', '')
        gt_bbox_list = get_gt_bbox(img_filename)
        if vis:       
            im2show = vis_bbox(im2show, gt_bbox_list, (0,255,0))
        
        # if there is det
        if inds.numel() > 0:
            cls_scores = scores[:,j][inds]
            _, order = torch.sort(cls_scores, 0, True)
            if args.class_agnostic:
                cls_boxes = pred_boxes[inds, :]
            else:
                cls_boxes = pred_boxes[inds][:, j * 4:(j + 1) * 4]
            
            cls_dets = torch.cat((cls_boxes, cls_scores.unsqueeze(1)), 1)
            # cls_dets = torch.cat((cls_boxes, cls_scores), 1)
            cls_dets = cls_dets[order]
            # keep = nms(cls_dets, cfg.TEST.NMS, force_cpu=not cfg.USE_GPU_NMS)
            keep = nms(cls_boxes[order, :], cls_scores[order], cfg.TEST.NMS)
            cls_dets = cls_dets[keep.view(-1).long()]
            
            # bbox pred / gt and iou calculation
            pred_bbox_list = get_pred_bbox(pascal_classes[j], cls_dets.cpu().numpy(), 0.9)
        
            # IoU calculation
            if len(pred_bbox_list):
                for bbox_obj in pred_bbox_list:
                    iou_list.append(get_iou(gt_bbox_list[0]['bbox'], bbox_obj['bbox']))
            else:
                iou_list.append(0.0)
            
            if vis:
                im2show = vis_bbox(im2show, gt_bbox_list, (0,255,0))
                im2show = vis_bbox(im2show, pred_bbox_list, (0,0,255))
                
                
    misc_toc = time.time()
    nms_time = misc_toc - misc_tic
    
    #if webcam_num == -1:
    sys.stdout.write('im_detect: {:d}/{:d} {:.3f}s {:.3f}s   \n' \
                       .format(num_images + 1, len(imglist), detect_time, nms_time))
    sys.stdout.flush()
    
    if vis:
        img_output_path = '/home/martin/JupyterLab/output/faster_rcnn_img'
        result_path = os.path.join(img_output_path, imglist[num_images][:-4] + "_det.jpg")
        # PIXEL value conversion 
        im2show = cv2.convertScaleAbs(im2show, alpha=(255.0))
        cv2.imwrite(result_path, im2show)

Loaded Photo: 299 images.
im_detect: 299/299 0.095s 0.001s   
im_detect: 298/299 0.101s 0.002s   
im_detect: 297/299 0.101s 0.004s   
im_detect: 296/299 0.098s 0.001s   
im_detect: 295/299 0.100s 0.003s   
im_detect: 294/299 0.103s 0.004s   
im_detect: 293/299 0.098s 0.007s   
im_detect: 292/299 0.108s 0.002s   
im_detect: 291/299 0.103s 0.005s   
im_detect: 290/299 0.104s 0.001s   
im_detect: 289/299 0.099s 0.002s   
im_detect: 288/299 0.099s 0.003s   
im_detect: 287/299 0.098s 0.004s   
im_detect: 286/299 0.100s 0.004s   
im_detect: 285/299 0.099s 0.007s   
im_detect: 284/299 0.098s 0.004s   
im_detect: 283/299 0.099s 0.004s   
im_detect: 282/299 0.100s 0.004s   
im_detect: 281/299 0.099s 0.004s   
im_detect: 280/299 0.099s 0.004s   
im_detect: 279/299 0.097s 0.005s   
im_detect: 278/299 0.099s 0.008s   
im_detect: 277/299 0.104s 0.002s   
im_detect: 276/299 0.098s 0.004s   
im_detect: 275/299 0.104s 0.001s   
im_detect: 274/299 0.100s 0.002s   
im_detect: 273/299 0.097s 0.007s   
im